In [1]:
%run -i "../util/file_utils.ipynb"
%run -i "../util/lang_utils.ipynb"

In [2]:
%run -i "../util/util_simple_classifier.ipynb"

In [3]:
(train_df, test_df) = load_train_test_dataset_pd()

In [4]:
import gensim


In [21]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

In [6]:
vec_bank = model['bank']
print(vec_bank)
print(vec_bank.shape)

[ 2.19726562e-02  1.34765625e-01 -5.78613281e-02  5.56640625e-02
  9.91210938e-02 -1.40625000e-01 -3.03649902e-03  1.87988281e-02
  2.53906250e-01 -4.88281250e-02 -1.63574219e-02 -1.33666992e-02
  6.25000000e-02  6.07910156e-02 -9.22851562e-02  3.12500000e-01
  1.38282776e-04 -1.34765625e-01 -4.32128906e-02  1.16699219e-01
  2.22656250e-01 -9.81445312e-02  4.51660156e-02 -2.23388672e-02
  5.17578125e-02 -2.41210938e-01 -1.11328125e-01  9.71679688e-02
  2.28515625e-01 -1.08642578e-02 -4.02832031e-02 -1.83105469e-02
  3.10546875e-01  3.88183594e-02 -2.85156250e-01 -2.06054688e-01
  3.69140625e-01 -5.24902344e-02  1.30859375e-01  1.51367188e-01
  1.59179688e-01 -2.36328125e-01  7.47070312e-02 -5.54199219e-02
 -8.64257812e-02 -2.28515625e-01  2.44140625e-03  8.11767578e-03
 -1.62109375e-01  1.46484375e-01  1.40625000e-01 -3.82995605e-03
  1.09375000e-01  1.14257812e-01 -1.19140625e-01 -7.12890625e-02
 -3.53515625e-01  2.00195312e-01  5.76171875e-02 -3.67187500e-01
 -2.42919922e-02 -3.97949

In [7]:
print(model.most_similar(['bank'], topn=10))

[('banks', 0.7440759539604187), ('banking', 0.690161406993866), ('Bank', 0.6698698401451111), ('lender', 0.6342284679412842), ('banker', 0.6092953085899353), ('depositors', 0.6031531691551208), ('mortgage_lender', 0.5797975659370422), ('depositor', 0.5716427564620972), ('BofA', 0.5714625120162964), ('Citibank', 0.5589520335197449)]


In [8]:
# Define a sentence vector function, by concatenating word vectors
def get_word_vectors(sentence, model):
    word_vectors = []
    for word in sentence:
        try:
            word_vector = model[word.lower()] # retrieve the word vector from the model
            word_vectors.append(word_vector)
        except KeyError:
            continue
    return word_vectors # list of word vectors for the sentence

# Define a sentence vector function, by averaging word vectors
def get_sentence_vector(word_vectors):
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size) # return a zero vector if no word vectors found
    sentence_vector = np.mean(word_vectors, axis=0)
    return sentence_vector

In [9]:
def vectorize(sentence):
    word_vectors = get_word_vectors(sentence, model)
    return get_sentence_vector(word_vectors)

In [10]:
(X_train, X_test, y_train, y_test) = create_train_test_data(train_df, test_df, vectorize)
clf = train_classifier(X_train, y_train)
test_classifier(test_df, clf)

              precision    recall  f1-score   support

           0       0.54      0.58      0.56       160
           1       0.55      0.51      0.53       160

    accuracy                           0.54       320
   macro avg       0.54      0.54      0.54       320
weighted avg       0.54      0.54      0.54       320



In [22]:
# Train your own Word2Vec model on your corpus
import gensim
from gensim.models import Word2Vec
from datasets import load_dataset
from gensim import utils

In [23]:
train_dataset = load_dataset('rotten_tomatoes', split='train')
print(len(train_dataset))

8530


In [24]:
class RottenTomatosCorpus:
    def __init__(self, sentences):
        self.sentences = sentences
    def __iter__(self):
        for sentence in self.sentences:
            yield utils.simple_preprocess(
                gensim.parsing.preprocessing.remove_stopwords(sentence))

In [25]:
sentences = train_dataset['text']
corpus = RottenTomatosCorpus(sentences)

In [26]:
model2 = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=2, workers=12)
model2.train(corpus_iterable=corpus, total_examples=model2.corpus_count, epochs=100)
model2.save("../data/word2vec_rotten_tomatoes.model")

In [27]:
word1 = "great"
words = model2.wv.most_similar(word1, topn=10)
print(f"Top 10 words similar to '{word1}': {words}")

Top 10 words similar to 'great': [('board', 0.3842310607433319), ('escapist', 0.37991952896118164), ('labute', 0.36852070689201355), ('tim', 0.36755242943763733), ('throwaway', 0.36137086153030396), ('deliver', 0.35612714290618896), ('routine', 0.3476513624191284), ('countless', 0.34745416045188904), ('sake', 0.3391719162464142), ('light', 0.33512717485427856)]


In [28]:
(analogy_score, word_list) = model2.wv.evaluate_word_analogies('../data/questions-words.txt')
print(f"Analogy score: {analogy_score}")

Analogy score: 0.0016863406408094434


In [29]:
(analogy_score, word_list) = model.evaluate_word_analogies('../data/questions-words.txt')
print(f"Pretrained model analogy score: {analogy_score}")

Pretrained model analogy score: 0.7401448525607863
